In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
dataframe = pd.read_csv('drive/MyDrive/DATA 310/Project 2/qsar_fish_toxicity.csv', delimiter = ';' ,names = ['CIC0','SM1_Dz(Z)','GATS1i','NdsCH','NdssC','MLOGP','LC50 [-LOG(mol/L)]'])

In [ ]:
dataframe

,CIC0,SM1_Dz(Z),GATS1i,NdsCH,NdssC,MLOGP,LC50 [-LOG(mol/L)]
0,3.260,0.829,1.676,0,1,1.453,3.770
1,2.189,0.580,0.863,0,0,1.348,3.115
2,2.125,0.638,0.831,0,0,1.348,3.531
3,3.027,0.331,1.472,1,0,1.807,3.510
4,2.094,0.827,0.860,0,0,1.886,5.390
...,...,...,...,...,...,...,...
903,2.801,0.728,2.226,0,2,0.736,3.109
904,3.652,0.872,0.867,2,3,3.983,4.040
905,3.763,0.916,0.878,0,6,2.918,4.818
906,2.831,1.393,1.077,0,1,0.906,5.317


In [ ]:
X = dataframe.iloc[:,:-1].values
y = dataframe.iloc[:,-1].values

**SVR**

In [ ]:
def Project2_SVR (X, y, k, kernel_type, degree, C_start, C_end, C_tests, eps_start, eps_end, eps_tests, random_state=123):
  import numpy as np
  from sklearn.model_selection import KFold
  from sklearn.preprocessing import StandardScaler, PolynomialFeatures
  from sklearn.pipeline import Pipeline
  import matplotlib.pyplot as plt
  from sklearn.svm import SVR

  scale = StandardScaler()

  R2train_avg = []
  R2test_avg = []
  c_value = []
  eps_value = []

  kf = KFold(n_splits=k, random_state=123, shuffle=True)

  poly = PolynomialFeatures(degree = degree)
  pipe = Pipeline([['Scaler',scale],['Poly Feats',poly]])

  for c in np.linspace(C_start,C_end,C_tests):
   
    for e in np.linspace (eps_start, eps_end, eps_tests):
      test_model = SVR(kernel = kernel_type, degree = degree, C = c, epsilon = e)
      R2train_raw = []
      R2test_raw = []
     
      for idxtrain, idxtest in kf.split(X):
        Xtrain = X[idxtrain]
        Xtest = X[idxtest]
        ytrain = y[idxtrain]
        ytest = y[idxtest]
        Xpolytrain = pipe.fit_transform(Xtrain)
        Xpolytest = pipe.transform(Xtest)

        #Regression
        test_model.fit(Xpolytrain,ytrain)
        R2train_raw.append(test_model.score(Xpolytrain,ytrain))
        R2test_raw.append(test_model.score(Xpolytest,ytest))
      R2test_avg.append(np.mean(R2test_raw))
      R2train_avg.append(np.mean(R2train_raw))
      c_value.append(c)
      eps_value.append(e)
  
  return R2train_avg, R2test_avg, c_value, eps_value

In [ ]:
k = 10
kernel_type = 'linear'
degree = 2

C_start = 0.01
C_end = 10
C_tests = 10

eps_start = 0.001
eps_end = 1
eps_tests = 10

R2train, R2test, C_value, EPS_value = Project2_SVR(X, y, k, kernel_type, degree, C_start, C_end, C_tests, eps_start,eps_end, eps_tests)

In [ ]:
# SVR
idx_max = np.argmax(R2test)
print('Optimal C Value:',C_value[idx_max])
print('Optimal Epsilon Value:',EPS_value[idx_max])
print('R^2 Value at that Point:',R2test[idx_max])

Optimal C Value: 1.12
Optimal Epsilon Value: 0.778
R^2 Value at that Point: 0.5735712684961712
